# Cortex Game SAS Demo-Round 2_Amount

## Create library

link for reference: https://documentation.sas.com/doc/en/pgmsascdc/v_014/lestmtsglobal/n1nk65k2vsfmxfn1wu17fntzszbp.htm?fromDefault=

First create a new folder under "Files(Home)" called "Results" in your ODA.

Then create 2 libraries. Copy and paste Below code, remmember to change paths to where the "u3984293" and "Results" folder located on your ODA.

1. 'cortex' library is where all raw dataset needed inside.
2. 'results library' is where all interim and final output will be saved. This step important because you don't have authority to do any edit/change to the 'cortex' library, so please restore any interim dataset to the 'work' or 'results' library.

In [1]:
libname cortex '/home/u58717790/my_shared_file_links/u39842936/Cortex Data Sets';
libname results '/home/u58717790/Results';

SAS Connection established. Subprocess id is 11920

0 0

5                                                          The SAS System                         Tuesday, July 13, 2021 10:55:00 PM

24         ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
24       ! ods graphics on / outputfmt=png;
25         
26         libname cortex '//home/u58717790/my_shared_file_links/u39842936';
27         libname results '/home/u58717790/Output';
28         
29         
30         ods html5 (id=saspy_internal) close;ods listing;
31         

6                                                          The SAS System                         Tuesday, July 13, 2021 10:55:00 PM

32         


# Step1-2 Merge and DataPartition

link for reference: https://documentation.sas.com/doc/en/pgmsascdc/9.4_3.3/statug/statug_surveyselect_toc.htm

In [2]:
/* merge dataset hist and target_rd2*/
DATA model_rd2_amt; 
   MERGE cortex.hist cortex.target_rd2;
   BY ID;
   if Gavethisyear=1;
run;

data results.model_rd2_amt; /*drop na*/
set model_rd2_amt;
if not cmiss(of _numeric_);
run;

0 0

7                                                          The SAS System                         Tuesday, July 13, 2021 10:55:00 PM

35         ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
35       ! ods graphics on / outputfmt=png;
36         
37         /* merge dataset hist and target_rd1*/
38         DATA model_rd1;
39            MERGE cortex.hist cortex.target_rd1;
40            BY ID;
41         run;
42         
43         data results.model_rd1; /*drop na*/
44         set model_rd1;
45         if not cmiss(of _numeric_);
46         run;
47         
48         
49         ods html5 (id=saspy_internal) close;ods listing;
50         

8                                                          The SAS System                         Tuesday, July 13, 2021 10:55:00 PM

51         


In [3]:
/* data partition*/
proc surveyselect data=results.model_rd2_amt rate=0.6 
out=  donor_select outall
method=srs
seed =1234; 
run;

data results.train_rd2_amt results.test_rd2_amt; 
set donor_select; 
if selected =1 then output results.train_rd2_amt; 
else output results.test_rd2_amt; 
run;

0 32384


Selection Method,Simple Random Sampling
Input Data Set,MODEL_RD1
Random Number Seed,1234
Sampling Rate,0.6
Sample Size,201801
Selection Probability,0.600002
Sampling Weight,0
Output Data Set,DONOR_SELECT


# Step3 Linear Regression Model 
link for reference: https://documentation.sas.com/doc/en/pgmsascdc/9.4_3.3/statug/statug_glmselect_syntax01.htm

In [4]:
/* glm model*/
ods graphics off;
proc glmselect data=results.train_rd2_amt testdata=results.test_rd2_amt;
model amtThisYear=Age Salary Seniority GaveLastYear contact/ selection=none;
title 'Regression of donation this year '
'Predictors';
code file='/home/u58717790/Results/regression_2_amt.sas';
run;

0 41787


# Step4 Decision Tree Model
link for reference:  https://support.sas.com/documentation/solutions/emtmsas/94/emprcref.pdf

In [ ]:
/* decision tree model*/
proc arboretum data= results.train_rd2_amt;
target AmtThisYear / level=interval;
input Age Salary / level=interval;
input GaveLastYear Seniority contact/level=nominal;
score data=results.test_rd2_amt role=valid OUT=_NULL_ OUTFIT=results.DT_stat_rd2_amount;
code file='/home/u58717790/Results/decisiontree_2_amt.sas';
quit;

Other modeling method:  https://support.sas.com/documentation/solutions/emtmsas/94/emprcref.pdf

logistic regression: https://documentation.sas.com/doc/en/pgmsascdc/9.4_3.3/statug/statug_logistic_gettingstarted.htm

HP Forest: https://documentation.sas.com/doc/en/pgmsascdc/9.4_3.3/emhpprcref/emhpprcref_hpforest_syntax.htm

Gradient Boosting: https://video.sas.com/detail/video/5754395446001/training-gradient-boosting-models-in-model-studio

# Step5 Scoring the data
link for reference: https://documentation.sas.com/doc/en/pgmsascdc/9.4_3.3/statug/statug_plm_toc.htm

In [ ]:
/* scoring the data*/

/*Predict amount give given contact*/

DATA contact_rd2;
   MERGE cortex.score cortex.score_rd2_contact;
   BY ID;
run;

data results.contact_rd2; 
/*drop na*/
set contact_rd2;
if not cmiss(of _numeric_);
run; 

data results.amtcontact (keep= id p_amtthisyear rename=(p_amtthisyear=AmtContact));
set results.contact_rd2;
%include '/home/u58717790/Results/regression_2_amt.sas';
run;


/*Predict amount give given NOT contact*/ 

DATA nocontact_rd2;
   MERGE cortex.score cortex.SCORE_RD2_NOCONTACT;
   BY ID;
run;

data results.nocontact_rd2; /*drop na*/
set nocontact_rd2;
if not cmiss(of _numeric_);
run;


data results.amtnoncontact (keep= id p_amtthisyear rename=(p_amtthisyear=AmtNoContact));
set results.nocontact_rd2;
%include '/home/u58717790/Results/regression_2_amt.sas';
run;

# Step7 Exporting CSV

link for reference:

https://documentation.sas.com/doc/en/pgmsascdc/v_014/proc/p1nd17xr6wof4sn19zkmid81p926.htm
https://documentation.sas.com/doc/en/pgmsascdc/v_014/proc/n0qarv79909fdhn13c60hfubqwdq.htm

In [ ]:
/* Export data*/

DATA results.rd2_output_amt;
   MERGE results.amtcontact results.amtnoncontact ;
   BY ID;
run; 

proc export data=results.rd2_output_amt
outfile="/home/u58717790/Results/Round2 Output amt.csv" dbms=csv
replace;
run;